In [1]:
import pickle as pkl 

In [16]:
with open("results_llama.pickle", "rb") as f:
    pred = pkl.load(f)

In [17]:
pred_tables={}
for k in pred:
    for i in k: 
        pred_tables[i] = k[i]

In [19]:
with open("/home/turning/Jainit/TANQ/EVALUATION_OF_MODELS/gemini/wikitables/test.data", "r") as f:
    lines = f.readlines()

In [20]:
len(lines), len(pred_tables)

(2000, 2000)

In [21]:
idx_to_table = {i: line for i, line in enumerate(lines)}

In [25]:
import json

In [26]:
import re
import pandas as pd
def extract_table(raw_text):
    """
    Extracts a table from a raw text string which may be in one of several formats:
    - Plain text with code fences (e.g., starting with ```text and ending with ```)
    - A JSON string that contains a "result" key with a table string, possibly wrapped in triple quotes.
    
    The table rows are expected to be separated by the literal "<NEWLINE>" token,
    and each row is a pipe-separated list of fields.
    """
    text = raw_text.replace("Final Answer:", "").strip()
    
    # --- Step 1. If the input is JSON, extract the "result" field ---
    try:
        parsed = json.loads(text)
        if isinstance(parsed, dict) and "result" in parsed:
            text = parsed["result"]
    except json.JSONDecodeError:
        # Not valid JSON, so assume it's a plain string
        pass

    # --- Step 2. Remove markdown code fences if present ---
    # Remove starting fence like ```text (or any language) and the ending ```
    text = re.sub(r'^```(?:[^\n]*)\n', '', text)
    text = re.sub(r'\n```$', '', text)

    # --- Step 3. Remove extra triple quotes if present ---
    text = text.strip()
    if text.startswith('"""') and text.endswith('"""'):
        text = text[3:-3].strip()

    # --- Step 4. Split the string into rows using the <NEWLINE> token ---
    if "<NEWLINE>" in text:
        rows = text.split("<NEWLINE>")
    else:
        rows = text.split("\n")
    records = []
    for row in rows:
        row = row.strip()
        if not row:
            continue
        # Remove any leading/trailing pipes and split on "|"
        # Then filter out any empty strings after stripping whitespace
        parts = [p.strip() for p in row.strip('|').split("|") if p.strip()]
        if len(parts) >= 2:
            key = parts[0]
            # In case there are additional tokens, join them with " | " to preserve inner separators.
            value = " | ".join(parts[1:])
            records.append((key, value))
    
    # --- Step 5. Create a DataFrame from the records ---
    df = pd.DataFrame(records, columns=["Field", "Value"])
    return df


In [29]:
input_tables = {}
for i, table in pred_tables.items():
    input_tables[i] = [extract_table(idx_to_table[i]).to_markdown(index = False), extract_table(table).to_markdown(index=False)]

In [34]:
print(input_tables[12][0])

| Field            | Value                              |
|:-----------------|:-----------------------------------|
| title            | youngstown–warren regional airport |
| subtitle         | passenger traffic                  |
| year             | 2015                               |
| total passengers | 129,000                            |


In [35]:
print(input_tables[12][1])

| Field            | Value                              |
|:-----------------|:-----------------------------------|
| category         | statistics                         |
| name             | youngstown–warren regional airport |
| year             | 2015                               |
| total passengers | 129,000                            |


In [36]:
json.dump(input_tables, open("input_tables_for_score.json", "w"), indent=2)